In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\geesi\Anaconda3\envs\dlnd\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./MNIST_data', validation_size=0)

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


Below I'm plotting an example image from the MNIST dataset. These are 28x28 grayscale images of handwritten digits.

In [11]:
inputs_ = tf.placeholder(tf.float32, (None, 784))
targets_ = tf.placeholder(tf.int64, [None])
keep_prob = tf.placeholder(tf.float32)

input_layer = tf.reshape(inputs_, [-1, 28, 28, 1])
conv1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=keep_prob)

logits = tf.layers.dense(inputs=dropout, units=10) 

cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=targets_, logits=logits)
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), targets_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()

## Training

In [12]:
# Create the session
sess = tf.Session()

Here I'll write a bit of code to train the network. I'm not too interested in validation here, so I'll just monitor the training loss. 

Calling `mnist.train.next_batch(batch_size)` will return a tuple of `(images, labels)`. We're not concerned with the labels here, we just need the images. Otherwise this is pretty straightfoward training with TensorFlow. We initialize the variables with `sess.run(tf.global_variables_initializer())`. Then, run the optimizer and get the loss with `batch_cost, _ = sess.run([cost, opt], feed_dict=feed)`.

In [20]:
epochs = 20
batch_size = 100
sess.run(tf.global_variables_initializer())
for ii in range(2000):      
    batch = mnist.train.next_batch(batch_size, shuffle=True)
    feed = {inputs_: batch[0], targets_: batch[1], keep_prob: 0.4}
    batch_cost, _ = sess.run([cross_entropy, train_step], feed_dict=feed)

    #print("Epoch: {}/{}...".format(e+1, epochs),
    #        "Training loss: {:.4f}".format(batch_cost))
    if (ii % 100 == 99):
        feed = {inputs_: mnist.test.images, targets_: mnist.test.labels, keep_prob:1}
        loss, valid_accuracy = sess.run([cross_entropy, accuracy], feed_dict=feed)
            
        print('Epoch {:<3} - Validation Accuracy: {}, Loss:{}'.format(ii, valid_accuracy, loss))

Epoch 99  - Validation Accuracy: 0.964900016784668, Loss:0.11545532196760178
Epoch 199 - Validation Accuracy: 0.978600025177002, Loss:0.06902900338172913
Epoch 299 - Validation Accuracy: 0.9797999858856201, Loss:0.05451254919171333
Epoch 399 - Validation Accuracy: 0.9846000075340271, Loss:0.05311987176537514
Epoch 499 - Validation Accuracy: 0.9815999865531921, Loss:0.05718529596924782
Epoch 599 - Validation Accuracy: 0.9847000241279602, Loss:0.04626205191016197
Epoch 699 - Validation Accuracy: 0.9886000156402588, Loss:0.03195573762059212
Epoch 799 - Validation Accuracy: 0.9879999756813049, Loss:0.03617949038743973
Epoch 899 - Validation Accuracy: 0.989799976348877, Loss:0.028688764199614525
Epoch 999 - Validation Accuracy: 0.9846000075340271, Loss:0.042116839438676834
Epoch 1099 - Validation Accuracy: 0.9900000095367432, Loss:0.030429242178797722
Epoch 1199 - Validation Accuracy: 0.991599977016449, Loss:0.028416229411959648
Epoch 1299 - Validation Accuracy: 0.9905999898910522, Loss:0.0

In [9]:
sess.close()